# How to use a cohort

This notebook shows how to use a cohort saved from Data Explorer.

It uses a cohort saved in the [Terra Notebooks Playground workspace](https://app.terra.bio/#workspaces/help-gatk/Terra%20Notebooks%20Playground/data).

## Setup

In [1]:
import os

import pandas as pd
import firecloud.api as fapi

In [2]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']

## Retrieve cohort SQL query

In [3]:
# Hard-code instead of use WORKSPACE_NAMESPACE/WORKSPACE_NAME, since other workspaces
# won't have the 1000g_americans cohort.
ws_namespace = 'help-gatk'
ws_name = 'Terra Notebooks Playground'
cohort_query = fapi.get_entity(ws_namespace, ws_name, 'cohort', '1000g_americans').json()['attributes']['query']
cohort_query

'SELECT DISTINCT t1.participant_id FROM (SELECT participant_id FROM `verily-public-data.human_genome_variants.1000_genomes_participant_info` WHERE  ((Super_Population_Description = "American"))) t1'

## Create pandas dataframe of cohort participant ids

In [4]:
participant_ids = pd.read_gbq(
    cohort_query,
    dialect='standard')
participant_ids.head()

,participant_id
0,HG01433
1,HG01445
2,HG01452
3,HG01473
4,HG01482


## See what tables are available to join against

In [5]:
bq_table_entities = fapi.get_entities(ws_namespace, ws_name, 'BigQuery_table').json()
bq_tables = list(map(lambda e: e['attributes']['table_name'], bq_table_entities))
bq_tables

['verily-public-data.human_genome_variants.1000_genomes_participant_info',
 'verily-public-data.human_genome_variants.1000_genomes_sample_info']

## Join cohort participant ids against sample_info table

In [6]:
sample_info = pd.read_gbq("SELECT * FROM `verily-public-data.human_genome_variants.1000_genomes_sample_info`",
                          dialect="standard")
print("sample_info has %d rows" % len(sample_info.index))

sample_info_americans = participant_ids.join(sample_info, lsuffix='_L', rsuffix='_R')
print("sample_info_americans has %d rows\n" % len(sample_info_americans.index))

sample_info_americans.head()

sample_info has 3500 rows
sample_info_americans has 535 rows



,participant_id_L,sample_id,participant_id_R,In_Low_Coverage_Pilot,LC_Pilot_Platforms,LC_Pilot_Centers,In_High_Coverage_Pilot,HC_Pilot_Platforms,HC_Pilot_Centers,In_Exon_Targetted_Pilot,...,exome_mapped_cram,exome_mapped_crai,exome_mapped_csra,exome_unmapped_bam,exome_unmapped_bai,exome_unmapped_bas,wgs_high_cov_bam,wgs_high_cov_bai,wgs_high_cov_cram,wgs_high_cov_crai
0,HG01433,HG00144,HG00144,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,HG01445,HG00147,HG00147,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,HG01452,HG00248,HG00248,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,HG01473,HG00411,HG00411,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,HG01482,HG00600,HG00600,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


# Provenance

In [7]:
import datetime
print(datetime.datetime.now())
!pip3 freeze

2021-05-19 23:09:19.788995
absl-py==0.12.0
anyio==3.1.0
argon2-cffi==20.1.0
arrow==1.1.0
arviz==0.11.2
asn1crypto==0.24.0
astroid==2.5.6
astunparse==1.6.3
async-generator==1.10
attrs==21.2.0
backcall==0.2.0
bagit==1.8.1
bgzip==0.3.5
binaryornot==0.4.4
biopython==1.78
bleach==3.3.0
bokeh==2.3.1
brewer2mpl==1.4.1
bx-python==0.8.11
CacheControl==0.11.7
cachetools==4.2.2
certifi==2020.12.5
cffi==1.14.5
cftime==1.4.1
chardet==4.0.0
cli-builder==0.1.5
click==7.1.2
colorama==0.4.4
confuse==1.4.0
cookiecutter==1.7.2
crcmod==1.7
cryptography==3.4.7
cwltool==1.0.20190228155703
cycler==0.10.0
Cython==0.29.23
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.3
entrypoints==0.3
fastinterval==0.1.1
fastprogress==1.0.0
filelock==3.0.12
firecloud==0.16.25
flatbuffers==1.12
future==0.18.2
gast==0.3.3
ggplot==0.11.5
gitdb==4.0.7
GitPython==3.1.17
google-api-core==1.26.3
google-auth==1.30.0
google-auth-oauthlib==0.4.4
google-cloud-bigquery==2.15.0
google-cloud-bigquery-datatransfer==3.1.1
goo

Copyright 2019 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.